### Домашка

1. Скачать [датасет по ссылке](https://raw.githubusercontent.com/tyqiangz/multilingual-sentiment-datasets/refs/heads/main/data/english/test.csv) (просто запустить строку ниже)

In [ ]:
!wget https://raw.githubusercontent.com/tyqiangz/multilingual-sentiment-datasets/refs/heads/main/data/english/test.csv

2. Открыть csv и сохранить текст в переменную (макс. балл == 1)

In [ ]:
import pandas as pd

df = pd.read_csv("test.csv")
texts = df["text"].astype(str)

3. Извлечь столбец с текстами (макс. балл == 1)

In [ ]:
df = pd.read_csv('test.csv') 
texts = df['text'].tolist()

4. Построить Bag-of-Words (макс. балл == 3)

In [ ]:
import gensim
import gensim.downloader as api
from gensim import corpora
from gensim.models import TfidfModel
import numpy as np
from pprint import pprint
import pandas as pd
import string
import re
import string

emoji_pattern = re.compile(
    "[" 
    "\U0001F600-\U0001F64F"
    "\U0001F300-\U0001F5FF"
    "\U0001F680-\U0001F6FF"
    "\U0001F700-\U0001F77F"
    "\U0001F780-\U0001F7FF"
    "\U0001F800-\U0001F8FF"
    "\U0001F900-\U0001F9FF"
    "\U0001FA00-\U0001FA6F"
    "\U0001FA70-\U0001FAFF"
    "]+"
)

df = pd.read_csv("test.csv")
texts = df["text"].astype(str)

tokenized_docs = [emoji_pattern.sub('', doc).lower().split() for doc in texts]
dictionary = corpora.Dictionary(tokenized_docs)

bow_corpus = [dictionary.doc2bow(doc) for doc in tokenized_docs]

def bow_to_dense(bow_corpus, dictionary):
    dense_vectors = []
    for doc in bow_corpus:
        dense_vec = [0] * len(dictionary)
        for idx, freq in doc:
            dense_vec[idx] = freq
        dense_vectors.append(dense_vec)
    return dense_vectors

dense_vectors = bow_to_dense(bow_corpus, dictionary)

import matplotlib.pyplot as plt
import seaborn as sns

# Визуализация
matrix = np.array(dense_vectors)
words = list(dictionary.token2id.keys())
doc_names = [f"Doc {i+1}" for i in range(len(texts))]

print(f"Matrix shape: {matrix.shape}")

plt.figure(figsize=(25, 6))

plt.subplot(1, 2, 1)
sns.heatmap(matrix,
            annot=True,
            fmt='d',
            xticklabels=words,
            yticklabels=doc_names,
            cmap='Blues',
            cbar_kws={'label': 'Word Frequency'})
plt.title('Bag-of-Words Matrix Heatmap')
plt.xticks(rotation=45)
plt.ylabel('Documents')
plt.xlabel('Words')

5. Построить TF-IDF (макс. балл == 5)

In [ ]:
tfidf_model = TfidfModel(bow_corpus)
tfidf_corpus = tfidf_model[bow_corpus]

def tfidf_to_dense(tfidf_corpus, dictionary):
    dense_vectors = []
    for doc in tfidf_corpus:
        dense_vec = [0.0] * len(dictionary)
        for idx, score in doc:
            dense_vec[idx] = score
        dense_vectors.append(dense_vec)
    return dense_vectors

tfidf_dense = tfidf_to_dense(tfidf_corpus, dictionary)
tfidf_matrix = np.array(tfidf_dense)
words = list(dictionary.token2id.keys())
doc_names = [f"Док {i+1}" for i in range(len(tfidf_corpus))]

plt.figure(figsize=(40, 20))

plt.subplot(2, 3, 1)
sns.heatmap(tfidf_matrix,
            annot=True,
            fmt='.3f',
            xticklabels=words,
            yticklabels=doc_names,
            cmap='YlOrRd',
            cbar_kws={'label': 'TF-IDF Вес'})
plt.xticks(rotation=45)

plt.subplot(2, 3, 2)

tf_dense = bow_to_dense(bow_corpus, dictionary)
tf_matrix = np.array(tf_dense)

doc_idx = 0
x_pos = np.arange(len(words))
width = 0.35

plt.bar(x_pos - width/2, tf_matrix[doc_idx], width, label='TF', alpha=0.7, color='blue')
plt.bar(x_pos + width/2, tfidf_matrix[doc_idx], width, label='TF-IDF', alpha=0.7, color='red')
plt.xticks(x_pos, words, rotation=45)
plt.legend()
plt.grid(True, alpha=0.3)

# Задача 2.2: Проанализируйте веса TF-IDF (макс. балл == 3)

df = pd.read_csv("test.csv")
texts = df["text"].astype(str)

tokenized_docs = [emoji_pattern.sub('', doc).lower().split() for doc in texts]
vocab = sorted(set(word for doc in tokenized_docs for word in doc))

tf = pd.DataFrame([
    [doc.count(word) for word in vocab] 
    for doc in texts
], columns=vocab)

df_values = (tf > 0).sum(axis=0)

N = len(tf)
idf = np.log((N) / (df_values + 1)) + 1  
idf = pd.Series(idf, index=vocab)

tfidf = tf * idf

result = pd.concat([
    tf.T.add_prefix("TF_doc_"),
    df_values.rename("DF"),
    idf.rename("IDF"),
    tfidf.T.add_prefix("TFIDF_doc_")
], axis=1)

output_path = "result.csv"
result.to_csv(output_path, encoding="utf-8-sig")
print(f"Файл сохранён: {output_path}")